## **Import Library**

In [ ]:
# Mengimpor library untuk membaca file model dan tokenizer yang disimpan
import pickle

# Mengimpor library untuk manipulasi array dan operasi numerik
import numpy as np

# Mengimpor TensorFlow untuk memuat dan menjalankan model deep learning
import tensorflow as tf

# Mengimpor fungsi untuk padding input teks agar memiliki panjang yang seragam
from keras.preprocessing.sequence import pad_sequences


from sklearn.preprocessing import LabelEncoder

# Menonaktifkan peringatan agar output lebih bersih
import warnings
warnings.filterwarnings("ignore")

## **Memuat Model dan Alat Preprocessing untuk Inference**

In [ ]:
# Memuat model LSTM dari file .h5 yang berada di dalam folder ../Models
lstm_model = tf.keras.models.load_model("lstm_sentiment_model.keras")

# Membuka dan memuat model SVM dari file pickle
with open("svm_sentiment_model.pkl", "rb") as f:
    svm_model = pickle.load(f)

# Membuka dan memuat model Random Forest dari file pickle
with open("rf_sentiment_model.pkl", "rb") as f:
    rf_model = pickle.load(f)

# Memuat kembali TF-IDF Vectorizer untuk mentransformasi teks ke bentuk vektor numerik
with open("vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

# Memuat tokenizer untuk mengubah teks menjadi urutan token angka untuk model LSTM
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

## **Preprocessing Untuk Data Baru**

In [ ]:
# Data teks baru yang akan digunakan untuk prediksi sentimen
new_texts = [
    "Pengiriman cepat.",  # Kemungkinan sentimen positif
    "Aplikasi lemot sering eror hanya loading saja.",  # Kemungkinan sentimen negatif
    "Tambah metode pembayaran M-banking!."  # Kemungkinan netral
]

# Preprocessing untuk Model LSTM:
# Mengubah teks menjadi urutan token numerik
new_sequences = tokenizer.texts_to_sequences(new_texts)

# Melakukan padding pada urutan token agar seragam panjangnya (maxlen=100)
new_padded = pad_sequences(new_sequences, maxlen=100)

# Preprocessing untuk Model SVM dan Random Forest:
# Mengubah teks menjadi representasi vektor numerik dengan metode TF-IDF
new_tfidf = vectorizer.transform(new_texts)

## **Perform Sentiment Prediction**

In [ ]:
# Melakukan prediksi menggunakan model LSTM
# Hasil prediksi berupa probabilitas untuk setiap kelas, sehingga kita ambil nilai maksimum (argmax)
lstm_preds = np.argmax(lstm_model.predict(new_padded), axis=1)

# Melakukan prediksi menggunakan model SVM
# Model SVM secara langsung menghasilkan label prediksi dari data TF-IDF
svm_preds = svm_model.predict(new_tfidf)

# Melakukan prediksi menggunakan model Random Forest
# Sama seperti SVM, Random Forest memberikan prediksi berbasis data TF-IDF
rf_preds = rf_model.predict(new_tfidf)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


## **Menampilkan Hasil Prediksi Sentimen**

In [ ]:
#Mapping label numerik ke label string agar hasil prediksi lebih mudah dibaca
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Menampilkan hasil prediksi dari ketiga model untuk setiap input teks
print("\nSentiment Prediction Results: ")
for i, text in enumerate(new_texts):
    print(f"Text: {text}")  # Menampilkan teks input
    print(f"LSTM Prediction: {label_map[lstm_preds[i]]}")  # Hasil prediksi model LSTM
    print(f"SVM Prediction: {label_map[svm_preds[i]]}")    # Hasil prediksi model SVM
    print(f"Random Forest Prediction: {label_map[rf_preds[i]]}")  # Hasil prediksi model RF
    print("-" * 50)  # Garis pemisah antar hasil


Sentiment Prediction Results: 
Text: Pengiriman cepat.
LSTM Prediction: positive
SVM Prediction: positive
Random Forest Prediction: positive
--------------------------------------------------
Text: Aplikasi lemot sering eror hanya loading saja.
LSTM Prediction: negative
SVM Prediction: negative
Random Forest Prediction: negative
--------------------------------------------------
Text: Tambah metode pembayaran M-banking!.
LSTM Prediction: neutral
SVM Prediction: neutral
Random Forest Prediction: neutral
--------------------------------------------------
